In [305]:
import requests
import numpy as np
import pandas as pd

In [306]:
res = requests.get("https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/sdg_07_40?format=JSON&lang=EN")
raw = res.json()

In [307]:
raw

{'version': '2.0',
 'class': 'dataset',
 'label': 'Share of renewable energy in gross final energy consumption by sector',
 'source': 'ESTAT',
 'updated': '2024-04-05T23:00:00+0200',
 'value': {'684': 29.62,
  '685': 31.367,
  '686': 32.07,
  '687': 32.657,
  '688': 32.448,
  '689': 31.437,
  '690': 31.867,
  '691': 31.187,
  '692': 35.152,
  '693': 33.167,
  '694': 31.856,
  '695': 34.913,
  '696': 36.953,
  '697': 35.776,
  '698': 36.572,
  '699': 38.042,
  '700': 45.015,
  '701': 41.389,
  '702': 44.076,
  '399': 22.553,
  '400': 24.353,
  '401': 26.276,
  '402': 28.144,
  '403': 28.788,
  '404': 31.039,
  '405': 31.205,
  '406': 31.552,
  '407': 32.734,
  '408': 32.665,
  '409': 33.55,
  '410': 33.497,
  '411': 33.37,
  '412': 33.136,
  '413': 33.784,
  '414': 33.755,
  '415': 36.545,
  '416': 34.573,
  '417': 33.758,
  '637': 24.873,
  '638': 26.607,
  '639': 25.358,
  '640': 23.241,
  '641': 35.972,
  '642': 37.454,
  '643': 39.835,
  '644': 36.562,
  '38': 1.916,
  '39': 2.325,


In [313]:
def parse_df_from_eurostat(values: dict[str: str], tables: list[str], rows: list[str], cols: list[str], label:str="", col_label:str="") -> list[pd.DataFrame]:
	"""
	Given a dict of indexed scaler values and the label lists, parse into a list of dataframes
	Note: Across all tables, the size (row x col) should be the same!

	:param values: A dict with indexed scaler values, ie {'572': 145.5} where the index is the cell that value occupies
	:param tables: A list of the table labels
	:param rows: A list of row labels
	:param cols: A list of column labels 
	:param label: The optional column label for the row keys
	:param col_label: The optional label for the column header group
	:returns: A dataframe with multiindexes for each table
	"""
	calc_index = lambda i, j, k: k + (j * len(cols)) + (i * (len(cols) * len(rows)))
	data_3d = []
	for i in range(len(tables)):
		data_2d = []
		for j in range(len(rows)):
			builder_row = []
			for k in range(len(cols)):
				if str(calc_index(i, j, k)) in values:
					builder_row.append(values[str(calc_index(i, j, k))])
				else:
					builder_row.append(np.NaN)
			data_2d.append(builder_row)
		data_3d.append(data_2d)
	

	dataframes = []
	for table in data_3d:
		df = pd.DataFrame(table)
		dataframes.append(df)
	
	df = pd.concat(dataframes, axis=1)
	cols = [l[0] for l in cols]
	df.columns = pd.MultiIndex.from_product([tables, cols], names=["table", col_label])
	if label:
		df[label] = [i[0] for i in rows]

	return df

In [314]:
# tables = []
# for stat in raw["dimension"]["statinfo"]["category"]["label"].items():
# 	for unit in raw["dimension"]["unit"]["category"]["label"].items():
# 		tables.append(stat[0] + unit[0])

values = raw["value"]
rows = list(raw["dimension"]["geo"]["category"]["label"].items())
cols = list(raw["dimension"]["time"]["category"]["label"].items())
tables = [i[0] for i in raw["dimension"]["nrg_bal"]["category"]["label"].items()]

In [315]:
df = parse_df_from_eurostat(values, tables, rows, cols, label="country", col_label="time")
df.head()

table    REN                                                                  \
time    2004    2005    2006    2007    2008    2009    2010    2011    2012   
0      9.605  10.182  10.778  11.749  12.552  13.850  14.405  14.547  16.002   
1        NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2      1.916   2.325   2.658   3.140   3.611   4.746   6.004   6.302   7.086   
3      9.231   9.173   9.415   9.098  10.345  12.005  13.927  14.152  15.837   
4      6.773   7.113   7.362   7.895   8.674   9.977  10.513  10.945  12.814   

table          ... REN_HEAT_CL                                          \
time     2013  ...        2014    2015    2016    2017    2018    2019   
0      16.659  ...      19.929  20.307  20.400  20.819  21.604  22.432   
1         NaN  ...         NaN     NaN     NaN     NaN     NaN     NaN   
2       7.671  ...       7.751   7.942   8.247   8.200   8.290   8.319   
3      18.898  ...      28.518  28.903  29.990  29.854  33.250  35.423   
4      13.927  ...      19.528  19.787  19.879  19.728  20.642  22.632   

table                            country  
time     2020    2021    2022             
0      22.983  22.932  24.858  EU27_2020  
1         NaN     NaN     NaN       EU28  
2       8.447   9.225  10.444         BE  
3      37.178  30.024  31.668         BG  
4      23.535  24.347  25.802         CZ  

[5 rows x 77 columns]

In [316]:
df.xs(tables[0], level=0, axis=1).head()

time,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,9.605,10.182,10.778,11.749,12.552,13.850,14.405,14.547,16.002,16.659,17.416,17.820,17.978,18.411,19.096,19.887,22.038,21.893,23.047
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.916,2.325,2.658,3.140,3.611,4.746,6.004,6.302,7.086,7.671,8.038,8.060,8.744,9.136,9.472,9.929,13.000,13.007,13.759
3,9.231,9.173,9.415,9.098,10.345,12.005,13.927,14.152,15.837,18.898,18.050,18.261,18.760,18.695,20.581,21.545,23.319,19.447,19.095
4,6.773,7.113,7.362,7.895,8.674,9.977,10.513,10.945,12.814,13.927,15.074,15.070,14.926,14.799,15.139,16.239,17.303,17.671,18.195
